In [1]:
import os
from glob import glob

# Avoids warnings
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "1"
import tensorflow as tf
import pandas as pd
import numpy as np

from ScalableLib.mixture.layers import create_models
from ScalableLib.classifier import Multiband as multiband

2024-10-02 14:40:30.111333: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-02 14:40:30.111370: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-02 14:40:30.112378: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-02 14:40:31.075006: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Classifier mode only should ignore physical parameters listed in the input.

In [2]:
# To see if the system recognises the GPU
device = 1
devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.set_visible_devices(devices[device], 'GPU')
tf.config.experimental.set_memory_growth(device=devices[device], enable=True)

device_name = tf.config.experimental.get_device_details(devices[device])['device_name']
print("Using {}".format(device_name))

Using NVIDIA GeForce RTX 2080 Ti


Find the different folds and train a model using the stored data.

In [3]:
survey = 'PanStarrs'
path = os.path.join('../../02_CreateRecords/', survey, 'Folds/Fold_*',)
folds = glob(path)
folds.sort()
folds

['../../02_CreateRecords/PanStarrs/Folds/Fold_1',
 '../../02_CreateRecords/PanStarrs/Folds/Fold_2',
 '../../02_CreateRecords/PanStarrs/Folds/Fold_3',
 '../../02_CreateRecords/PanStarrs/Folds/Fold_4',
 '../../02_CreateRecords/PanStarrs/Folds/Fold_5',
 '../../02_CreateRecords/PanStarrs/Folds/Fold_6',
 '../../02_CreateRecords/PanStarrs/Folds/Fold_7']

Create folder results

In [4]:
if not os.path.exists('./Results'):
    os.mkdir('./Results')


Define the arguments for all the models.

In [5]:
train_args = {
            'hidden_size_bands':[64, 64, 64],
            'hidden_size_central':[64, 64, 64],
            'fc_layers_bands':[128,128],
            'fc_layers_central':[128,128], # Neurons of each layer
            'regression_size':[128, 128],#each element is a layer with that size.
            'buffer_size':10000,
            'epochs':1000,
            'num_threads':7,
            'batch_size':512,
            'dropout':0.35,
            'lr':[[5e-3]*5, 2.5e-3], # [[band1, band2], central]
            'val_steps':50,
            'max_to_keep':0, # Not Used 
            'steps_wait':500, 
            'use_class_weights':False,# Not Used
            'mode' : 'classifier'
            }
loss_weights = {'Class':1.0}

callbacks_args = {'patience': 20,
                  'mode':'max',
                  'restore_best_weights':True,
                  'min_delta': 0.001
                 }
train_args_specific={
                    'phys_params': [],
                    'use_output_bands' : True,  # Working
                    'use_output_central' : False, # Not used
                    'use_common_layers' : False, # NOT Working
                    'bidirectional_central' : False,# Working
                    'bidirectional_band' : False,# Not Working
                    'layer_norm_params' : None, # Used to normalyze common layers
                    'use_gated_common' : False, # Working
                    'l1':0.0,
                    'l2':0.0,
                    'N_skip' : 2, # Cannot be greater than the number of timesteps
                    'use_raw_input_central': False,
                    'train_steps_central' : 1,
                    'print_report' : True,
                    'loss_weights_central' : loss_weights,
                    'callbacks_args':callbacks_args    
                    }



In [6]:
sauces = []
for fold in folds:
    tf.keras.backend.clear_session()
    # Set the fold path
    base_dir = fold+'/'
    
    # Set the save path for this fold. Create folder if needed
    path_results_fold = fold.replace('../../02_CreateRecords/'+survey+'/', './').replace('/Folds/', '/Results/')
    if not os.path.exists(path_results_fold):
        os.mkdir(path_results_fold)    
        
    train_args_specific['save_dir'] = path_results_fold
    train_args_specific['metadata_pre_path'] = base_dir+'metadata_preprocess.json'  
    train_args_specific['path_scalers'] =  os.path.join(fold,'scalers.pkl')
    # Define the train args
    train_args = {**train_args, **train_args_specific}
    
    train_files = base_dir+'train/*.tfrecord'
    val_files = base_dir+'val/*.tfrecord'
    test_files = base_dir+'test/*.tfrecord'
    
    new = multiband.Network()    
    new.train(train_args, train_files, val_files, test_files)
    # Create the Mixture of Experts model, from the original multiband model
    model_base = create_models(new)
    model = model_base.creat_split_models(train_args)

    # Define the early stopping callback
    es = tf.keras.callbacks.EarlyStopping(monitor='accuracy', **callbacks_args)
    # Fit the model to the original data
    model.fit(new.dataset_train, validation_data=new.dataset_val, epochs=300,callbacks = [es] )

    # Store the alpha coefficients per fold
    sauces_ = {i.name:i for i in model.layers if 'Sauce_' in i.name}
    scales = {key:tf.nn.softmax(sauces_[key].scale).numpy() for key in sauces_.keys()}
    sauces.append(scales)
    # Test
    dfs = []
    for batch in new.dataset_test:
        prediction = model(batch[0])
        y_pred = prediction['Class'].numpy().argmax(axis=1)
        y_pred = [new.trans[i] for i in y_pred]
        ID = batch[0]['ID'].numpy()

        y_true = batch[1]['Class'].numpy().argmax(axis=1)
        y_true = [new.trans[i] for i in y_true]

        df = pd.DataFrame(np.array([ID, y_pred, y_true]).transpose(), columns=['ID', 'Class', 'Pred'])
        df.ID = df.ID.str.decode('UTF-8')
        dfs.append(df)
    dfs = pd.concat(dfs, axis=0)
    dfs.to_csv(path_results_fold+'/Classification_test.dat', index=False, index_label=False)
    
pd.DataFrame(sauces).to_csv(path_results_fold+'/sauces.dat', index=False, index_label=False)    

./Results/Fold_1/Models/20241002-1440
Epoch 1/300


I0000 00:00:1727894540.033931   30256 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


36/36 [==============================] - 92s 688ms/step - loss: 1.0702 - accuracy: 0.5323 - val_loss: 0.8616 - val_accuracy: 0.5845
Epoch 2/300
36/36 [==============================] - 3s 92ms/step - loss: 0.8918 - accuracy: 0.5884 - val_loss: 0.7822 - val_accuracy: 0.6447
Epoch 3/300
36/36 [==============================] - 3s 91ms/step - loss: 0.7923 - accuracy: 0.6599 - val_loss: 0.7165 - val_accuracy: 0.6866
Epoch 4/300
36/36 [==============================] - 3s 90ms/step - loss: 0.7161 - accuracy: 0.6942 - val_loss: 0.6530 - val_accuracy: 0.7226
Epoch 5/300
36/36 [==============================] - 3s 93ms/step - loss: 0.6410 - accuracy: 0.7454 - val_loss: 0.5842 - val_accuracy: 0.7675
Epoch 6/300
36/36 [==============================] - 3s 93ms/step - loss: 0.5802 - accuracy: 0.7762 - val_loss: 0.5531 - val_accuracy: 0.7874
Epoch 7/300
36/36 [==============================] - 3s 92ms/step - loss: 0.5505 - accuracy: 0.7919 - val_loss: 0.5474 - val_accuracy: 0.7871
Epoch 8/300
36/3